In [3]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam, SGD
from keras.preprocessing.image import ImageDataGenerator

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128 #128 - original batch size
num_classes = 10
epochs = 50
l = 40
num_filter = 12
compression = 0.5
dropout_rate = 0.2

In [7]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
#print (img_height, img_width, channel)

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 41s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 16  
# original is 12 
# 64, 128 and 256 are giving above 1MM+ Parameters, hence going with 32.
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Fourth_Block = add_denseblock(Third_Transition, num_filter, dropout_rate)
Fourth_Transition = add_transition(Fourth_Block, num_filter, dropout_rate)

#Fifth_Block = add_denseblock(Fourth_Transition, num_filter, dropout_rate)
#Fifth_Transition = add_transition(Fifth_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Fourth_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [13]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_79 (Conv2D)              (None, 32, 32, 16)   432         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_79 (BatchNo (None, 32, 32, 16)   64          conv2d_79[0][0]                  
__________________________________________________________________________________________________
activation_79 (Activation)      (None, 32, 32, 16)   0           batch_normalization_79[0][0]     
__________________________________________________________________________________________________
conv2d_80 

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=SGD(lr=0.1),
              metrics=['accuracy'])

In [15]:
# Adding Image Augmentation

from keras.preprocessing.image import ImageDataGenerator
gen = ImageDataGenerator(rotation_range=8, width_shift_range=0.08, shear_range=0.3,
                         height_shift_range=0.1, zoom_range=0.1)

test_gen = ImageDataGenerator()

train_generator = gen.flow(x_train, y_train, batch_size=batch_size)
test_generator = test_gen.flow(x_test, y_test, batch_size=batch_size)


model.fit_generator(train_generator, steps_per_epoch=60000//batch_size, epochs=epochs, validation_data=test_generator, validation_steps=10000//batch_size)

Epoch 1/50
468/468 [==============================] - 182s 389ms/step - loss: 1.8750 - acc: 0.2875 - val_loss: 1.7092 - val_acc: 0.3697
Epoch 2/50
468/468 [==============================] - 158s 338ms/step - loss: 1.5923 - acc: 0.4028 - val_loss: 1.8160 - val_acc: 0.3973
Epoch 3/50
468/468 [==============================] - 158s 338ms/step - loss: 1.4513 - acc: 0.4631 - val_loss: 1.4562 - val_acc: 0.4833
Epoch 4/50
468/468 [==============================] - 158s 337ms/step - loss: 1.3490 - acc: 0.5080 - val_loss: 1.6189 - val_acc: 0.4607
Epoch 5/50
468/468 [==============================] - 159s 339ms/step - loss: 1.2728 - acc: 0.5393 - val_loss: 1.5164 - val_acc: 0.5144
Epoch 6/50
468/468 [==============================] - 158s 337ms/step - loss: 1.2151 - acc: 0.5619 - val_loss: 1.2245 - val_acc: 0.5692
Epoch 7/50
468/468 [==============================] - 158s 337ms/step - loss: 1.1589 - acc: 0.5821 - val_loss: 1.3357 - val_acc: 0.5664
Epoch 8/50
468/468 [============================

In [16]:
# Model fit implimentation that came with original code
# Runnig for 50 epochs
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 125s 3ms/step - loss: 0.4903 - acc: 0.8304 - val_loss: 0.6809 - val_acc: 0.7951
Epoch 2/50
50000/50000 [==============================] - 125s 2ms/step - loss: 0.4800 - acc: 0.8352 - val_loss: 0.5856 - val_acc: 0.8196
Epoch 3/50
50000/50000 [==============================] - 125s 3ms/step - loss: 0.4737 - acc: 0.8369 - val_loss: 0.6372 - val_acc: 0.8050
Epoch 4/50
50000/50000 [==============================] - 125s 2ms/step - loss: 0.4706 - acc: 0.8370 - val_loss: 0.6932 - val_acc: 0.7999
Epoch 5/50
50000/50000 [==============================] - 125s 3ms/step - loss: 0.4696 - acc: 0.8393 - val_loss: 0.8737 - val_acc: 0.7634
Epoch 6/50
50000/50000 [==============================] - 125s 3ms/step - loss: 0.4604 - acc: 0.8414 - val_loss: 0.7492 - val_acc: 0.7833
Epoch 7/50
50000/50000 [==============================] - 125s 2ms/step - loss: 0.4572 - acc: 0.8441 - val_loss: 0.7024 - 

In [15]:
# Running for 25 epochs
#model.fit_generator(train_generator, steps_per_epoch=60000//batch_size, epochs=epochs//2, validation_data=test_generator, validation_steps=10000//batch_size)

Epoch 1/25
468/468 [==============================] - 268s 573ms/step - loss: 0.5027 - acc: 0.8271 - val_loss: 0.7950 - val_acc: 0.7809
Epoch 2/25
468/468 [==============================] - 268s 573ms/step - loss: 0.5016 - acc: 0.8292 - val_loss: 0.7699 - val_acc: 0.7894
Epoch 3/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4847 - acc: 0.8317 - val_loss: 0.6935 - val_acc: 0.8021
Epoch 4/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4812 - acc: 0.8325 - val_loss: 0.7994 - val_acc: 0.7820
Epoch 5/25
468/468 [==============================] - 268s 573ms/step - loss: 0.4766 - acc: 0.8355 - val_loss: 0.7693 - val_acc: 0.7950
Epoch 6/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4777 - acc: 0.8359 - val_loss: 0.8063 - val_acc: 0.7800
Epoch 7/25
468/468 [==============================] - 269s 574ms/step - loss: 0.4666 - acc: 0.8400 - val_loss: 0.7514 - val_acc: 0.7926
Epoch 8/25
468/468 [============================

In [17]:
# adding another training for 50 epochs
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=100,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/100
50000/50000 [==============================] - 125s 3ms/step - loss: 0.3485 - acc: 0.8793 - val_loss: 0.5949 - val_acc: 0.8373
Epoch 2/100
50000/50000 [==============================] - 125s 2ms/step - loss: 0.3448 - acc: 0.8802 - val_loss: 0.5180 - val_acc: 0.8501
Epoch 3/100
50000/50000 [==============================] - 125s 2ms/step - loss: 0.3491 - acc: 0.8794 - val_loss: 0.6392 - val_acc: 0.8243
Epoch 4/100
50000/50000 [==============================] - 125s 2ms/step - loss: 0.3469 - acc: 0.8790 - val_loss: 0.9834 - val_acc: 0.7659
Epoch 5/100
50000/50000 [==============================] - 125s 2ms/step - loss: 0.3456 - acc: 0.8797 - val_loss: 0.6603 - val_acc: 0.8241
Epoch 6/100
50000/50000 [==============================] - 125s 2ms/step - loss: 0.3424 - acc: 0.8798 - val_loss: 0.7536 - val_acc: 0.8094
Epoch 7/100
50000/50000 [==============================] - 125s 3ms/step - loss: 0.3430 - acc: 0.8812 - val_loss: 0.

In [18]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 14s 1ms/step
Test loss: 0.6045607993364334
Test accuracy: 0.8498


In [19]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:

from google.colab import files

files.download('DNST_model.h5')